In [ ]:
#관측 결과의 누적된 결핍 → 회로 구조 확장 구조


import numpy as np
from qiskit import QuantumCircuit, Aer, execute
import matplotlib.pyplot as plt
from collections import Counter

sim = Aer.get_backend('qasm_simulator')
shots = 1024
steps = 30

# 초기 조건
n = 2  # 시작 큐비트 수
U = [np.random.uniform(0, np.pi) for _ in range(n)]
dominant_counts = Counter()
trajectory = []
n_history = []

# 회로 실행 함수
def run_circuit(U):
    qc = QuantumCircuit(len(U), len(U))
    for i, theta in enumerate(U):
        qc.ry(theta, i)
    qc.barrier()
    qc.measure(range(len(U)), range(len(U)))
    result = execute(qc, sim, shots=shots).result()
    return result.get_counts()

# 큐비트 수 확장 조건: dominant 상태가 연속 5회 이상 등장
threshold = 5
last_dominant = None
dominant_streak = 0

for step in range(steps):
    counts = run_circuit(U)
    trajectory.append(counts)
    n_history.append(len(U))

    # 가장 많이 나온 상태
    dominant = max(counts.items(), key=lambda x: x[1])[0]

    if dominant == last_dominant:
        dominant_streak += 1
    else:
        dominant_streak = 1
        last_dominant = dominant

    # 누적 조건 만족 → 큐비트 추가
    if dominant_streak >= threshold:
        U.append(np.pi / 4)  # 새 큐비트 초기화
        dominant_streak = 0  # 리셋

# 시각화: 큐비트 수 변화
plt.figure(figsize=(10, 4))
plt.plot(n_history, marker='o')
plt.title("Qubit Count Over Time (Structure Expansion via Repetitive Observation)")
plt.xlabel("Step")
plt.ylabel("Qubit Count")
plt.grid(True)
plt.show()
